In [1]:
# Lab 06
# Date: 4/4/2024
# Jyosna Philip
# 23232019

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
import pandas as pd


In [38]:

df = pd.read_csv('titanic_train.csv')
df
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
# here we remove the columns that would probably have no affect on the output.

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [23]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)
X_train


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [24]:
df['Age'].isna().sum()
#checking how many null values are there in age column

177

In [12]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

#Simple imputer changes the null values. the default strategy is replacing with the mean value,
#  which is what happens in the first instance of simple imputer
#most frequesnt strategy replaces the null value with whatever occurs most
#here we dont fit the simpleImputers because we fit data after w e create pipeline

#the column transformer allows us to apply different imputation functions at the same 
# time to different columns of the dataset
#first parameter of this function is a list of transformers each being a 3-tuple,
#3-tuple should be in the order-(Name,transformer,list of column_indexes)
#name is not col name, its a name to call that specific transformation.
#by default whatever columns of a dataset that dont pass through this col transformer will be dropped.
# to prevent that, we have to put the remainder argument.


In [30]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])
#fitting simple_imputer is neccessary,without fitting you wont be able to transform another data
#here mean of first col is 7
#mean of sec col is 2+5/2=3.5 (because one col is nan)
#3rd col mean is 6

SimpleImputer()

In [32]:
X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
print(imp_mean.transform(X))
#the output shows that nan is replaced with whatevr value was obtained during fitting

[[ 7.   2.   3. ]
 [ 4.   3.5  6. ]
 [10.   3.5  9. ]]


In [13]:
df['Age'].isna().sum()
#we can see that the actual df has no effect, by declaring trf1, which is obvious
#as no data was sent through

177

In [36]:
#fit_transform is a function where fitting and transform occurs together
df_small = pd.read_csv('titanic_train - Copy.csv')
df_small.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df_small


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,NaN
4,0,3,male,35.0,0,0,8.0500,NaN
5,0,3,male,NaN,0,0,8.4583,NaN
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S


In [44]:
trf_small = ColumnTransformer([
    ('impute_age',SimpleImputer(),[3]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[7])
],remainder='passthrough')

In [45]:
new=trf_small.fit_transform(df_small)

print(new)


[[22.0 'S' 0 3 'male' 1 0 7.25]
 [38.0 'C' 1 1 'female' 1 0 71.2833]
 [26.0 'S' 1 3 'female' 0 0 7.925]
 [35.0 'S' 1 1 'female' 1 0 53.1]
 [35.0 'S' 0 3 'male' 0 0 8.05]
 [29.875 'S' 0 3 'male' 0 0 8.4583]
 [54.0 'S' 0 1 'male' 0 0 51.8625]
 [2.0 'S' 0 3 'male' 3 1 21.075]
 [27.0 'S' 1 3 'female' 0 2 11.1333]]


One Hot encoding is a method to represent categorical variables as numeric
if we map male and female to 1 and 0 respectively, there could be a bias as 1>0
for each category a column is created, where if the category is true for a particular row, the value in the row will be 1, or else 0.


In [51]:
trf_small = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,2])
],remainder='passthrough')

In [52]:
new=trf_small.fit_transform(df_small)

print(new)
#we can see that new columns have been added

[[0.0 1.0 0.0 1.0 0 22.0 1 0 7.25 'S']
 [1.0 0.0 1.0 0.0 1 38.0 1 0 71.2833 'C']
 [0.0 1.0 1.0 0.0 1 26.0 0 0 7.925 'S']
 [1.0 0.0 1.0 0.0 1 35.0 1 0 53.1 nan]
 [0.0 1.0 0.0 1.0 0 35.0 0 0 8.05 nan]
 [0.0 1.0 0.0 1.0 0 nan 0 0 8.4583 nan]
 [1.0 0.0 0.0 1.0 0 54.0 0 0 51.8625 'S']
 [0.0 1.0 0.0 1.0 0 2.0 3 1 21.075 'S']
 [0.0 1.0 1.0 0.0 1 27.0 0 2 11.1333 'S']]


In [14]:

# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')


In [15]:
# Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])


In [16]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)
clf = LogisticRegression()


In [17]:
## Pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('clf',clf)
])


In [21]:
pipe.isna().sum()

AttributeError: 'Pipeline' object has no attribute 'isna'

In [ ]:
# train
pipe.fit(X_train,y_train)
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')
# Predict
y_pred = pipe.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)
## Cross Validation
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()
## GridSearchCV 
# gridsearchcv
param_grid = {
    "clf__penalty": ['l1', 'l2'],
    "clf__C": [0.001, 0.01, 0.1, 1, 10, 100]
}
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)
grid.best_score_
grid.best_params_
## Export to a Pickle File
# export 
import pickle
pickle.dump(pipe,open('lr_pile.pkl','wb'))
